**Trabalho em Grupo**

Link Edmodo: https://new.edmodo.com/turnin/assignment:42942429:145606093

**Problema**

> Conversa com personagens do Star Wars: Crie um chatbot que, dado um texto que você digitou, gere uma resposta no estilo "Star Wars". A resposta deve conter 100 palavras.

Para criar o modelo de geração de texto, você pode utilizar o dataset https://www.kaggle.com/xvivancos/star-wars-movie-scripts


Nome dos integrantes:

* Aluno 1: Aline Cristini - 183132
* Aluno 2: Camila Rodrigues - 183143
* Aluno 3: Rafael Gimenes Leite - 101634


In [1]:
#imports necessarios
import numpy as np
import pandas as pd
import string, os 
import keras
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from keras.layers.core import Activation, Dense, Dropout
from keras.layers.wrappers import TimeDistributed
from keras.layers.core import Dense, Activation
import keras.utils as kutils
import pickle
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import string


Using TensorFlow backend.


In [2]:
#loading datasets
df_ep_VI = pd.read_table("../input/SW_EpisodeVI.txt",header=0, escapechar='\\',delim_whitespace=True,)
df_ep_IV = pd.read_table('../input/SW_EpisodeIV.txt',header=0, escapechar='\\',delim_whitespace=True)
df_ep_V = pd.read_table("../input/SW_EpisodeV.txt",header=0, escapechar='\\',delim_whitespace=True)

In [3]:
#checking if all have same structure
df_ep_VI.columns

Index(['character', 'dialogue'], dtype='object')

In [4]:
df_ep_V.columns

Index(['character', 'dialogue'], dtype='object')

In [5]:
df_ep_IV.columns

Index(['character', 'dialogue'], dtype='object')

In [6]:
#checking sizes
print(df_ep_VI.shape)
print(df_ep_V.shape)
print(df_ep_IV.shape)


(674, 2)
(839, 2)
(1010, 2)


In [7]:
#join the datasets, to work with only one.
df = pd.concat([df_ep_IV, df_ep_V, df_ep_VI])

In [8]:
#checking size
df.shape

(2523, 2)

In [9]:
#getting the lines 
all_head_lines = []
for filename in df:
    all_head_lines.extend(list(df.dialogue.values))
    break


In [10]:
len(all_head_lines)

2523

In [11]:
all_head_lines[:5]

["Did you hear that?  They've shut down the main reactor.  We'll be destroyed for sure.  This is madness!",
 "We're doomed!",
 "There'll be no escape for the Princess this time.",
 "What's that?",
 'I should have known better than to trust the logic of a half-sized thermocapsulary dehousing assister...']

In [12]:
#creating the corpus
def lower_text(txt):
    txt = "".join(b for b in txt if b not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt 

corpus = [lower_text(line) for line in all_head_lines]


In [13]:
len(corpus)

2523

In [14]:
corpus[:5]

['did you hear that  theyve shut down the main reactor  well be destroyed for sure  this is madness',
 'were doomed',
 'therell be no escape for the princess this time',
 'whats that',
 'i should have known better than to trust the logic of a halfsized thermocapsulary dehousing assister']

In [15]:
#creating sents
sents = [[wd.lower() for wd in word_tokenize(sent) if not wd in string.punctuation]  for sent in all_head_lines]
x = []
y = []
print(sents[:10])

[['did', 'you', 'hear', 'that', 'they', "'ve", 'shut', 'down', 'the', 'main', 'reactor', 'we', "'ll", 'be', 'destroyed', 'for', 'sure', 'this', 'is', 'madness'], ['we', "'re", 'doomed'], ['there', "'ll", 'be', 'no', 'escape', 'for', 'the', 'princess', 'this', 'time'], ['what', "'s", 'that'], ['i', 'should', 'have', 'known', 'better', 'than', 'to', 'trust', 'the', 'logic', 'of', 'a', 'half-sized', 'thermocapsulary', 'dehousing', 'assister', '...'], ['hurry', 'up', 'come', 'with', 'me', 'what', 'are', 'you', 'waiting', 'for', 'get', 'in', 'gear'], ['artoo', 'artoo-detoo', 'where', 'are', 'you'], ['at', 'last', 'where', 'have', 'you', 'been'], ['they', "'re", 'heading', 'in', 'this', 'direction', 'what', 'are', 'we', 'going', 'to', 'do', 'we', "'ll", 'be', 'sent', 'to', 'the', 'spice', 'mines', 'of', 'kessel', 'or', 'smashed', 'into', 'who', 'knows', 'what'], ['wait', 'a', 'minute', 'where', 'are', 'you', 'going']]


In [16]:
for sent in sents:
    for i in range(1, len(sent)):
        x.append(sent[:i])
        y.append(sent[i])

In [17]:
print(x[:5])
print(y[:5])


[['did'], ['did', 'you'], ['did', 'you', 'hear'], ['did', 'you', 'hear', 'that'], ['did', 'you', 'hear', 'that', 'they']]
['you', 'hear', 'that', 'they', "'ve"]


In [18]:
#train,test split
text = [i for sent in x for i in sent]
text += [i for i in y]
#unknow
text.append('UNK') 
words = list(set(text))      
word_indexes = {word: index for index, word in enumerate(words)}      
max_features = len(word_indexes)

x = [[word_indexes[i] for i in sent] for sent in x]
y = [word_indexes[i] for i in y]

In [19]:
print(x[:5])
print(y[:5])

[[766], [766, 1776], [766, 1776, 26], [766, 1776, 26, 2532], [766, 1776, 26, 2532, 390]]
[1776, 26, 2532, 390, 1091]


In [20]:
y = kutils.to_categorical(y, num_classes=max_features)
maxlen = max([len(sent) for sent in x])
print(maxlen)

138


In [21]:
x = pad_sequences(x, maxlen=maxlen)
x = pad_sequences(x, maxlen=maxlen)

for ya in y:
    for i in range(len(ya)):
        if ya[i] != 0:
            print(i)

1776
26
2532
390
1091
697
1440
492
1908
440
109
2360
1717
2700
262
2733
1382
6
2719
366
70
2360
1717
598
821
262
492
1958
1382
77
1865
2532
1646
1728
2162
1799
2361
924
68
492
1153
668
474
641
1675
1610
494
273
337
2735
38
1582
2709
2383
1776
44
262
2067
820
525
681
2536
2383
1776
1981
2536
1728
1776
2391
366
25
820
1382
577
2709
2383
109
1850
924
796
109
2360
1717
1683
924
492
1160
2170
668
468
2342
648
1586
3
2671
2709
474
2767
2536
2383
1776
1850
1669
287
824
2383
307
820
492
1908
1133
2383
1760
2611
1776
517
517
598
2611
676
273
1382
6
474
2202
1123
1213
1636
474
622
2542
1382
6
474
2202
1123
273
2536
6
492
1069
1961
1382
1123
1316
93
1776
1091
2759
1760
824
223
2795
1582
492
1069
1348
1061
2736
752
1862
6
303
262
23
2360
1717
2492
986
1491
1408
1605
109
1728
474
2618
1776
366
307
1370
820
1470
17
1865
1736
1776
2360
1717
361
262
462
2141
2449
1582
474
45
1266
1776
2264
618
668
491
1689
2735
2537
371
2505
1270
1776
2542
2709
824
2709
2383
1776
357
811
1348
679
307
2452
820
1470
679

In [22]:
embedding_size = 10
model = Sequential()  
# Add Input Embedding Layer
model.add(Embedding(max_features, embedding_size, input_length=maxlen))
# Add Hidden Layer 1 - LSTM Layer
model.add(LSTM(100))
model.add(Dropout(0.1))
    
# Add Output Layer
model.add(Dense(max_features, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [23]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 138, 10)           28010     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               44400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2801)              282901    
Total params: 355,311
Trainable params: 355,311
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.fit(x, y, epochs=30, verbose=5)

Instructions for updating:
Use tf.cast instead.
Epoch 1/30
Epoch 2/30
Epoch 3/30
Epoch 4/30
Epoch 5/30
Epoch 6/30
Epoch 7/30
Epoch 8/30
Epoch 9/30
Epoch 10/30
Epoch 11/30
Epoch 12/30
Epoch 13/30
Epoch 14/30
Epoch 15/30
Epoch 16/30
Epoch 17/30
Epoch 18/30
Epoch 19/30
Epoch 20/30
Epoch 21/30
Epoch 22/30
Epoch 23/30
Epoch 24/30
Epoch 25/30
Epoch 26/30
Epoch 27/30
Epoch 28/30
Epoch 29/30
Epoch 30/30


In [25]:
print("Saving model...")
model.save('shak-nlg.h5')

with open('shak-nlg-dict.pkl', 'wb') as handle:
    pickle.dump(word_indexes, handle)

with open('shak-nlg-maxlen.pkl', 'wb') as handle:
    pickle.dump(maxlen, handle)
print("Model Saved!")

Saving model...
Model Saved!


In [26]:
model = keras.models.load_model('shak-nlg.h5')
maxlen = pickle.load(open('shak-nlg-maxlen.pkl', 'rb'))
word_indexes = pickle.load(open('shak-nlg-dict.pkl', 'rb'))

In [27]:
#igual da aula
sample_seed = input()
sample_seed_vect = np.array([[word_indexes[c] if c in word_indexes.keys() else word_indexes['UNK'] for c in word_tokenize(sample_seed)]])

print(sample_seed_vect)
sample_seed_vect = pad_sequences(sample_seed_vect, maxlen=maxlen)
print(sample_seed_vect)

predicted = model.predict_classes(sample_seed_vect, verbose=0)
print(predicted)

def get_word_by_index(index, word_indexes):
    for w, i in word_indexes.items():
        if index == i:
            return w
        
    return None

for p in predicted:    
    print(get_word_by_index(p, word_indexes))

StdinNotImplementedError: raw_input was called, but this frontend does not support input requests.

In [28]:
sample_seed = input()
sample_seed_vect = [word_indexes[c] if c in word_indexes.keys() else word_indexes['UNK']  for c in word_tokenize(sample_seed)]

print(sample_seed_vect)
predicted = []

while len(sample_seed_vect) < 100:
    predicted = model.predict_classes(pad_sequences([sample_seed_vect], maxlen=maxlen, padding='pre'), verbose=0)
    sample_seed_vect.extend(predicted)

res = []

for p in sample_seed_vect:    
   res.append(get_word_by_index(p, word_indexes)) 

print(' '.join (res))

StdinNotImplementedError: raw_input was called, but this frontend does not support input requests.